In [27]:
import math
import time

import cv2 as cv
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.patches import Circle
from scipy.ndimage.filters import gaussian_laplace, rank_filter
from skimage import color, io, transform


# convert images to grayscale
# rescale the intensities to between 0 and 1 (simply divide them by 255 should do the trick)
def read_img(IMG_NAME):
    img = color.rgb2gray(io.imread(IMG_NAME)).astype(float)
    return img

# Creating the Laplacian filter
# Pay careful attention to setting the right filter mask size.


def laplace_filter(img, sigma):
    filtered = sigma * sigma * gaussian_laplace(img, sigma=sigma)
    return filtered


# nonmaximum suppression in scale space
# you may find functions scipy.ndimage.filters.rank_filter or scipy.ndimage.filters.generic_filter useful
def nonmax_suppress(scale_space, max_scale_space, n, threshold):

    h, w, n = scale_space.shape
    compare_scale_space = np.zeros((h, w, n))

    for i in range(h):
        for j in range(w):
            max_value = max(max_scale_space[i, j, :])
            max_index = np.where(max_scale_space[i, j, :] == max_value)[0][0]
            if max_value >= threshold and max_value == scale_space[i, j, max_index]:
                compare_scale_space[i, j, max_index] = max_value

    return compare_scale_space


def find_blobs(compare_scale_space, sigma, k):

    cx = []
    cy = []
    rad = []
    h, w, n = compare_scale_space.shape
    for layer in range(n):
        for i in range(h):
            for j in range(w):
                if compare_scale_space[i, j, layer] != 0:
                    cx.append(j)
                    cy.append(i)
                    blob_radius = sigma * k ** layer
                    rad.append(blob_radius)

    return cx, cy, rad


def detect_blobs_downsample(img_name, grey_img, org_img, sigma, n, k):
    print(grey_img.shape)
    h, w = grey_img.shape
    scale_space = np.zeros((h, w, n))
    max_scale_space = np.zeros((h, w, n))
    start_time = time.time()

    for i in range(n):
        scaling = 1.0 / (k ** (i))
        downsampled = transform.resize(
            grey_img, (int(scaling * h), int(scaling * w)), anti_aliasing=True)
        filtered = laplace_filter(downsampled, sigma)
        upsampled = transform.resize(
            np.square(filtered), (h, w),  anti_aliasing=True)
        scale_space[:, :, i] = upsampled
        max_scale_space[:, :, i] = rank_filter(
            scale_space[:, :, i], rank=-1, size=(5, 5))

    compare_scale_space = nonmax_suppress(
        scale_space, max_scale_space, n, 0.002)
    cx, cy, rad = find_blobs(compare_scale_space, sigma, k)

    # create keypoints
    kp = [cv.KeyPoint(cx[i], cy[i], rad[i]) for i in range(len(cx))]

    circle_img = cv.drawKeypoints(
        org_img, kp, org_img, flags=cv.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    plt.imsave(f'../Project1/tmp/{file_name}_circle_detect.jpg', circle_img)

    window = org_img
    color = (255, 0, 0)
    keypoints_num = len(cx)
    result = []
    for i in range(keypoints_num):
        radius = rad[i]

        start_x, start_y = math.floor(
            cx[i] - radius / 2), math.floor(cy[i] - radius / 2)
        end_x, end_y = math.floor(
            cx[i] + radius / 2), math.floor(cy[i] + radius / 2)

        if start_x < 0 or start_y < 0 or end_x >= w or end_y >= h:
            continue

        # 横向是 x 轴，纵向是 y 轴
        cv.rectangle(
            window, (start_x, start_y), (end_x, end_y), color, 1)

        result.append((start_x, start_y, end_x, end_y))

    plt.imsave(f'../Project1/tmp/{file_name}_window.jpg', window)

    print("sigma is ", sigma)
    print("Num of key points are ", len(kp))
    print("Time taken:", time.time() - start_time)


sigma = 4
n = 6
threshold = 1.25
raw_path = "../Project1/raw/"
file_name = "005-9-01.jpg"
img_name = raw_path+file_name
grey_img = read_img(img_name)
org_img = cv.imread(img_name)
window_points = detect_blobs_downsample(
    img_name, grey_img, org_img, sigma, n, threshold)


(4544, 3272)
sigma is  4
Num of key points are  27187
Time taken: 82.48546695709229
